In [1]:
import pandas as pd
import numpy as np
SGP = pd.read_csv('LabelBinarizedSGP.csv')

In [2]:
SGP.head()

,Unnamed: 0,school,sex,age,Medu,Fedu,traveltime,studytime,failures,higher,freetime,Dalc,Walc,G1,G2,G3
0,0,0,0,18,4,4,2,2,0,1,3,1,1,0,11,11
1,1,0,0,17,1,1,1,2,0,1,3,1,1,9,11,11
2,2,0,0,15,1,1,1,2,0,1,3,2,3,12,13,12
3,3,0,0,15,4,2,1,3,0,1,2,1,1,14,14,14
4,4,0,0,16,3,3,1,2,0,1,3,1,2,11,13,13


In [3]:
SGP.drop(['Unnamed: 0'], axis=1, inplace=True)

In [4]:
SGP.head()

,school,sex,age,Medu,Fedu,traveltime,studytime,failures,higher,freetime,Dalc,Walc,G1,G2,G3
0,0,0,18,4,4,2,2,0,1,3,1,1,0,11,11
1,0,0,17,1,1,1,2,0,1,3,1,1,9,11,11
2,0,0,15,1,1,1,2,0,1,3,2,3,12,13,12
3,0,0,15,4,2,1,3,0,1,2,1,1,14,14,14
4,0,0,16,3,3,1,2,0,1,3,1,2,11,13,13


In [5]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(SGP, test_size=0.2, random_state=25)
print ("Training and testing split was successful.")

Training and testing split was successful.


In [7]:
Y_train = X_train['G3']
Y_test = X_test['G3']

In [8]:
X_train.drop('G3', axis=1, inplace=True)
X_test.drop('G3', axis=1, inplace=True)

/home/dee/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
forest_reg.fit(X_train, Y_train)

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

### Performance Measure of the Model

In [10]:
#Using forest_reg.score
forest_reg.score(X_test, Y_test)

0.7925259961929931

In [11]:
#Using r2
from sklearn.metrics import r2_score
Y_pred = forest_reg.predict(X_test)
r2_score(Y_test, Y_pred)

0.7925259961929931

It can be observed that Random Forest Regressor is giving us an accuracy of upto 96% using two different accuracy metrics.

Let's see if it's for real or our model is simply overfitting.

### Validation for Forest Regression

In [12]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(forest_reg, X_test, Y_test, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation", scores.std())
display_scores(lin_rmse_scores)

Scores: [0.49302832 1.21939455 1.09509395 1.4655007  0.89657468 1.15358972
 2.4905977  1.00115318 1.42234909 2.94370254]
Mean: 1.418098442874834
Standard deviation 0.7067780895404386


In [13]:
some_data = X_test.iloc[:5]
some_labels = Y_test.iloc[:5]

In [14]:
print("Predictions:", forest_reg.predict(some_data))
print("Labels:", list(some_labels))

Predictions: [ 9.2 15.6 10.2 11.5 10.5]
Labels: [10, 15, 11, 12, 10]


### Hyperparameter Tuning

In [15]:
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [3, 10, 20, 30], 'max_features': [2, 4, 6, 8], 'max_depth': [2, 4, 6, 8, 10]},
{'bootstrap': [False, True], 'n_estimators': [3, 10], 'max_features': [2, 3, 4], 'max_depth': [2, 4, 6, 8, 10]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
scoring='neg_mean_squared_error')
grid_search.fit(X_train, Y_train)

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [3, 10, 20, 30], 'max_features': [2, 4, 6, 8], 'max_depth': [2, 4, 6, 8, 10]}, {'bootstrap': [False, True], 'n_estimators': [3, 10], 'max_features': [2, 3, 4], 'max_depth': [2, 4, 6, 8, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [16]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [17]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(grid_search.best_estimator_, X_test, Y_test, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation", scores.std())
display_scores(lin_rmse_scores)

Scores: [1.02683599 1.04595699 0.98599002 1.19512702 1.53707841 2.09884369
 2.46321668 1.08271042 0.88410526 2.90982171]
Mean: 1.5229686184361602
Standard deviation 0.6790247848802539


In [18]:
grid_search.best_params_

{'max_depth': 10, 'max_features': 6, 'n_estimators': 30}

In [19]:
grid_search.best_estimator_

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=6, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=30, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [20]:
feature_importances = grid_search.best_estimator_.feature_importances_

In [21]:
sorted(zip(feature_importances, X_train.columns), reverse=True)

[(0.5332687521436437, 'G2'),
 (0.25113543414218226, 'G1'),
 (0.044921958632845226, 'failures'),
 (0.02009453580090215, 'age'),
 (0.019454723388926242, 'higher'),
 (0.01808672849413321, 'school'),
 (0.01776301324025824, 'freetime'),
 (0.016905158718815188, 'Walc'),
 (0.016374308898748476, 'Medu'),
 (0.014911147195609119, 'Fedu'),
 (0.014211282857825332, 'studytime'),
 (0.014137223773236996, 'Dalc'),
 (0.010091131779952368, 'traveltime'),
 (0.008644600932921387, 'sex')]

In [22]:
X_train.drop(['higher', 'school', 'studytime'], axis=1, inplace=True)
X_test.drop(['higher', 'school', 'studytime'], axis=1, inplace=True)

/home/dee/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [23]:
forest_reg.fit(X_train, Y_train)

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [24]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(forest_reg, X_test, Y_test, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation", scores.std())
display_scores(forest_rmse_scores)

Scores: [0.45993311 1.24344435 1.77004998 1.86300006 1.18321596 1.61602551
 2.64444265 0.96116916 1.01223287 2.86033887]
Mean: 1.561385251459258
Standard deviation 0.715326136207041


In [25]:
from sklearn.model_selection import GridSearchCV
param_grid = [
{'n_estimators': [3, 10, 20, 30], 'max_features': [2, 4, 6, 8], 'max_depth': [2, 4, 6, 8, 10]},
{'bootstrap': [False, True], 'n_estimators': [3, 10], 'max_features': [2, 3, 4], 'max_depth': [2, 4, 6, 8, 10]},
]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=10,
scoring='neg_mean_squared_error')
grid_search.fit(X_train, Y_train)

/home/dee/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'n_estimators': [3, 10, 20, 30], 'max_features': [2, 4, 6, 8], 'max_depth': [2, 4, 6, 8, 10]}, {'bootstrap': [False, True], 'n_estimators': [3, 10], 'max_features': [2, 3, 4], 'max_depth': [2, 4, 6, 8, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [28]:
import numpy as np
from sklearn.model_selection import cross_val_score
scores = cross_val_score(grid_search.best_estimator_, X_test, Y_test, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation", scores.std())
display_scores(forest_rmse_scores)

Scores: [0.54347202 1.25824717 1.30669222 1.554512   1.29299562 1.27677248
 2.58481969 0.92194916 0.90090057 2.92027795]
Mean: 1.456063888757189
Standard deviation 0.7052796095526769


In [29]:
some_data = X_test.iloc[:5]
some_labels = Y_test.iloc[:5]
print("Predictions:", grid_search.best_estimator_.predict(some_data))
print("Labels:", list(some_labels))

Predictions: [ 9.47255284 15.33925587  9.18347702 11.29985281 10.28378829]
Labels: [10, 15, 11, 12, 10]
